In [35]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

In [36]:
#Load SMILES data
data = np.load("ohepsmiles.npz")
data = data["arr_0"]
data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
print("Dataset size: " + str(data.size()))

Dataset size: torch.Size([34131372, 1, 55])


In [37]:
#Initialize model parameters
input_size = np.shape(data)[1]
hidden_size = 256
num_layers = 3
dropout = 0.2
learning_rate = 0.001
epochs = 10
seq_length = 76

In [38]:
#Define model
model = torch.nn.Sequential(
    nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout),
    nn.Linear(hidden_size, input_size)
)   

In [50]:
#Initialize hidden states, initialize cell states
initial_hidden = Variable(torch.zeros(num_layers, 1, hidden_size))
initial_cell = Variable(torch.zeros(num_layers, 1, hidden_size))
print("Hidden state size: " + str(hidden.size()))
print("Cell state size: " + str(cell.size()))

Hidden state size: torch.Size([3, 1, 256])
Cell state size: torch.Size([3, 1, 256])


In [51]:
#Define loss function
loss_function = nn.CrossEntropyLoss()

#Define optimizer
model_optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [59]:
#Training
for e in range(epochs):
    #Get random order of SMILES molecules (shuffle data)
    shuffle = np.arange(np.shape(data)[0] / seq_length)
    random.shuffle(shuffle)
    
    for i in range(int(np.shape(data)[0] / seq_length)):
        #Zero gradients
        model.zero_grad()
        
        #Zero loss
        loss = 0
        
        #Input (does not include last character in SMILES)
        inp = data[int(shuffle[i] * seq_length) : int((shuffle[i] * seq_length) + seq_length - 2), :, :]
        #Target (does not include first character in SMILES)
        target = data[int((shuffle[i] * seq_length) + 1) : int((shuffle[i] * seq_length) + seq_length - 1), :, :]
        
        for s in range(seq_length):
            output, hidden = model((inp[s,:,:], (initial_hidden, initial_cell)))
        
            loss += loss_function(output, target[s,:,:])
        
        loss.backward()
        model_optimizer.step()
        
        print(loss)
    

AttributeError: 'tuple' object has no attribute 'size'